In [20]:
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
import random

chromedriver_path = r'C:\Users\luisf\Desktop\MDS_thesis\scraper_gn\chromedriver-win64\chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [26]:
search_query = "mañanera amlo"
start_date = "2018-12-03"
end_date = "2024-09-30"
max_pages = 50
limit=500 
count=0
current_page=1
url = f'https://www.google.com/search?q={search_query}+after:{start_date}+before:{end_date}&tbm=nws&sort=date'
driver.get(url)


In [27]:
#Second iteration with pagination... fully functional. 
news_data = []

while current_page <= max_pages and count < limit:
    print(f"Scraping page {current_page}")
    
    try:
        # Wait for the news results to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.SoaBEf'))
        )
    except Exception as e:
        print(f"Timeout while waiting for news articles on page {current_page}: {e}")
        # Optional: Take a screenshot for debugging
        driver.save_screenshot(f'timeout_page_{current_page}.png')
        break  # Exit if articles do not load in time

    # Select all news article blocks by targeting the SoaBEf class
    news_results = driver.find_elements(By.CSS_SELECTOR, 'div.SoaBEf')
    
    if not news_results:
        print(f"No articles found on page {current_page}.")
        # Optional: Take a screenshot to inspect the page
        driver.save_screenshot(f'no_articles_page_{current_page}.png')
        break  # Exit if no articles are found

    for news_div in news_results:
        if count >= limit:
            print(f"Reached the limit of {limit} articles.")
            break  # Stop if we've reached the limit

        try:
            # Extract the link to the article
            news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')

            # Extract the title of the article
            news_title = news_div.find_element(By.CSS_SELECTOR, 'a div[role="heading"]').text

            # Extract the source domain
            news_domain = news_div.find_element(By.CSS_SELECTOR, 'div').text

            # Extract the snippet description and the publication date
            news_description = news_div.find_element(By.CSS_SELECTOR, 'a div[role="heading"] + div').text
            news_date = news_div.find_element(By.CSS_SELECTOR, 'div.OSrXXb.rbYSKb.LfVVr > span').text

            # Append the collected information to the list
            news_data.append([news_link, news_domain, news_title, news_description, news_date])
            count += 1  # Increment the count of news articles

        except Exception as e:
            print(f"Error scraping an article on page {current_page}: {e}")
            # Optional: Take a screenshot for debugging
            driver.save_screenshot(f'error_article_page_{current_page}.png')
            continue  # Skip to the next article

    # Check if we've reached the limit after scraping the current page
    if count >= limit:
        print(f"Reached the limit of {limit} articles.")
        break

    # Attempt to locate the "Next" button and click it using the general selector
    try:
        # Find the "Next" button using its general ID selector
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#pnnext'))
        )
        print("Found 'Next' button using the '#pnnext' selector.")
        
        # Scroll to the "Next" button to ensure it's in view
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        
        # Introduce a random sleep to mimic human behavior
        time.sleep(random.uniform(2, 5))
        
        # Capture the current URL before clicking "Next"
        current_url = driver.current_url
        print(f"Current URL before clicking 'Next': {current_url}")
        
        # Click the "Next" button
        next_button.click()
        print(f"Clicked 'Next' button on page {current_page}.")
        
        # Increment the current page counter
        current_page += 1
        
        # Optional: Add a short sleep to wait for the next page to load
        time.sleep(random.uniform(2, 4))
        
        # Capture the new URL after clicking "Next"
        new_url = driver.current_url
        print(f"Navigated to new URL: {new_url}")
        
        # Verify that the URL has changed to prevent looping
        if new_url == current_url:
            print("URL did not change after clicking 'Next'. Possible loop detected.")
            break  # Exit the loop to prevent infinite cycling
        else:
            print(f"Successfully navigated to page {current_page}.")

    except Exception as e:
        print(f"No more pages available or error clicking 'Next' on page {current_page}: {e}")
        # Optional: Take a screenshot to inspect the state when the error occurred
        driver.save_screenshot(f'next_button_error_page_{current_page}.png')
        break  # Exit the loop if there's no next button or an error occurs

Scraping page 1
Found 'Next' button using the '#pnnext' selector.
Current URL before clicking 'Next': https://www.google.com/search?q=ma%C3%B1anera%20amlo+after:2018-12-03+before:2024-09-30&tbm=nws&sort=date
Clicked 'Next' button on page 1.
Navigated to new URL: https://www.google.com/search?q=ma%C3%B1anera+amlo+after:2018-12-03+before:2024-09-30&sca_esv=1f29f74ee812de91&tbm=nws&ei=WHb9ZsfwAu-Oxc8P05q18QU&start=10&sa=N&ved=2ahUKEwjH_KO4kPCIAxVvR_EDHVNNLV4Q8NMDegQIARAY&biw=958&bih=892&dpr=1
Successfully navigated to page 2.
Scraping page 2
Found 'Next' button using the '#pnnext' selector.
Current URL before clicking 'Next': https://www.google.com/search?q=ma%C3%B1anera+amlo+after:2018-12-03+before:2024-09-30&sca_esv=1f29f74ee812de91&tbm=nws&ei=WHb9ZsfwAu-Oxc8P05q18QU&start=10&sa=N&ved=2ahUKEwjH_KO4kPCIAxVvR_EDHVNNLV4Q8NMDegQIARAY&biw=958&bih=892&dpr=1
Clicked 'Next' button on page 2.
Navigated to new URL: https://www.google.com/search?q=ma%C3%B1anera+amlo+after:2018-12-03+before:2024-09

In [28]:
print(news_data)
print(len(news_data))

[['https://www.bbc.com/mundo/noticias-america-latina-47066862', 'BBC\nLas "mañaneras" de AMLO: cómo son las tempraneras conferencias con las que López Obrador marca la agenda política de México\nTodos los días, Andrés Manuel López Obrador conversa con periodistas. Es el primer presidente en la historia de México que lo hace. La estrategia, dicen...\n.\n31 ene 2019', 'Las "mañaneras" de AMLO: cómo son las tempraneras conferencias con las que López Obrador marca la agenda política de México', 'Todos los días, Andrés Manuel López Obrador conversa con periodistas. Es el primer presidente en la historia de México que lo hace. La estrategia, dicen...', '31 ene 2019'], ['https://www.infobae.com/mexico/2024/09/20/la-mananera-de-amlo-hoy-20-de-septiembre-en-vivo/', 'Infobae\nLa Mañanera de AMLO hoy 20 de septiembre | Rescate de PEMEX y CFE, beneplácito por la Guardia Nacional y cariño al PRD, entre lo relevante\nSe hablará de diversos temas y se responderán preguntas en la conferencia matutina 

In [29]:
df = pd.DataFrame(news_data, columns=["Link", "Domain", "Title", "Description", "Date"])
print(df)
print(len(df))

                                                  Link  \
0    https://www.bbc.com/mundo/noticias-america-lat...   
1    https://www.infobae.com/mexico/2024/09/20/la-m...   
2    https://www.gob.mx/presidencia/es/articulos/ve...   
3    https://www.eleconomista.com.mx/politica/amlo-...   
4    https://www.capital21.cdmx.gob.mx/noticias/?p=...   
..                                                 ...   
295  https://www.eleconomista.com.mx/politica/AMLO-...   
296  https://www.marca.com/mx/actualidad/2023/11/06...   
297  https://aristeguinoticias.com/1508/mexico/conf...   
298  https://www.informador.mx/mexico/AMLO-La-manan...   
299  https://www.eleconomista.com.mx/politica/YouTu...   

                                                Domain  \
0    BBC\nLas "mañaneras" de AMLO: cómo son las tem...   
1    Infobae\nLa Mañanera de AMLO hoy 20 de septiem...   
2    gob.mx\nVersión estenográfica. Conferencia de ...   
3    El Economista\nAMLO dirige su última mañanera ...   
4    Capital 

In [30]:

# Save the results to a CSV file
csv_file_path = "news_results.csv"
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(["Link", "Domain", "Title", "Description", "Date"])
    # Write the news data
    writer.writerows(news_data)

driver.quit()

print(f"Results saved to {csv_file_path}")

Results saved to news_results.csv
